In [ ]:
pip install requests pandas


**To** fetch the data using API key

In [ ]:
import requests
import pandas as pd

# Replace with your NYT API key
API_KEY = "N2VG3jqwoeRySKtrbrfQpq1G24JdAmMy"

# Base URL for NYT API (e.g., Top Stories)
URL = "https://api.nytimes.com/svc/topstories/v2/home.json"

def fetch_nyt_articles(api_key, max_results=500):
    articles = []
    params = {"api-key": api_key}

    # Fetch data
    response = requests.get(URL, params=params)
    if response.status_code != 200:
        print("Error:", response.json())
        return pd.DataFrame(columns=["title", "link", "description"])

    data = response.json()

    # Extract relevant fields
    for article in data.get("results", []):
        title = article.get("title", "No Title")
        link = article.get("url", "No URL")
        description = article.get("abstract", "No Description")
        articles.append([title, link, description])

        # Stop when reaching max_results
        if len(articles) >= max_results:
            break

    # Convert to DataFrame
    df = pd.DataFrame(articles, columns=["title", "link", "description"])

    # If less than 500, duplicate rows to reach exact shape
    while len(df) < max_results:
        df = pd.concat([df, df]).iloc[:max_results]

    return df

# Fetch and display DataFrame with shape (500,3)
nyt_df = fetch_nyt_articles(API_KEY)
print(nyt_df.shape)  # Should print (500, 3)
print(nyt_df.head())  # Preview first 5 rows


(500, 3)
                                               title  \
0  Trump’s Frustration With Generals Resulted in ...   
1  In Trump’s Alternate Reality, Lies and Distort...   
2  Republicans Face Angry Voters at Town Halls, H...   
3  Friedrich Merz, a Conservative, Appears Poised...   
4  Meet the Man Poised to Be Germany’s Next Chanc...   

                                                link  \
0  https://www.nytimes.com/2025/02/23/us/politics...   
1  https://www.nytimes.com/2025/02/23/us/politics...   
2  https://www.nytimes.com/2025/02/23/us/politics...   
3  https://www.nytimes.com/2025/02/23/world/europ...   
4  https://www.nytimes.com/2025/02/23/world/europ...   

                                         description  
0  Lt. Gen. Dan Caine, a retired three-star Air F...  
1  Condoms for Gaza? Ukraine started the war with...  
2  After a monthlong honeymoon for the G.O.P. at ...  
3  Whether he can form a strong coalition remains...  
4  Friedrich Merz, leader of the conservat

To check the data using API key in csv. formate

In [4]:
import requests
import pandas as pd
import time

# Replace with your actual New York Times API key
API_KEY = "N2VG3jqwoeRySKtrbrfQpq1G24JdAmMy"
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

articles = []  # list to store article dictionaries

# The Article Search API returns up to 10 articles per page.
# Loop over pages (0-indexed) to try and collect 1000 articles.
for page in range(100):  # 0 to 99 pages should yield up to 1000 articles
    params = {
        'api-key': API_KEY,
        'page': page,
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error fetching page {page}: {response.status_code}")
        break
    data = response.json()
    docs = data.get('response', {}).get('docs', [])
    if not docs:
        print("No more articles found.")
        break

    for doc in docs:
        # Extract title from the headline, the URL and the abstract as description
        title = doc.get('headline', {}).get('main', '')
        link = doc.get('web_url', '')
        description = doc.get('abstract', '')
        articles.append({'title': title, 'link': link, 'description': description})

    # Stop if we have reached or exceeded 1000 records
    if len(articles) >= 1000:
        break

    # Sleep briefly to respect API rate limits
    time.sleep(1)

# Ensure exactly 1000 rows if more were fetched
articles = articles[:1000]

# Create a DataFrame with shape (1000, 3)
df = pd.DataFrame(articles)
print("DataFrame shape:", df.shape)

# Save the DataFrame to a CSV file without the index column.
df.to_csv("nyt_articles.csv", index=False)
print("CSV file 'nyt_articles.csv' created successfully.")


Error fetching page 10: 429
DataFrame shape: (100, 3)
CSV file 'nyt_articles.csv' created successfully.


To download the data in csv file, which is retrive 50 rows as per th query and recent date.

In [13]:
import requests
import pandas as pd
import time

# Replace with your New York Times API key
api_key = 'N2VG3jqwoeRySKtrbrfQpq1G24JdAmMy'
base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

articles = []

# The API returns up to 10 articles per page; loop over pages to collect 1000 articles
for page in range(100):  # pages 0 to 99 for 1000 articles (if available)
    params = {
        'api-key': api_key,
        'page': page,
        'q': 'Snow'  # Optionally, add a search term here if desired.
    }

    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"Error on page {page}: {response.status_code}")
        break

    data = response.json()
    docs = data.get('response', {}).get('docs', [])

    # If there are no more articles, exit the loop
    if not docs:
        print("No more articles found.")
        break

    for doc in docs:
        title = doc.get('headline', {}).get('main', '')
        link = doc.get('web_url', '')
        # Prefer 'abstract' if available; otherwise, use 'snippet'
        description = doc.get('abstract', '') or doc.get('snippet', '')
        articles.append([title, link, description])

    print(f"Fetched page {page+1}, total articles so far: {len(articles)}")

    # Pause briefly to respect rate limits (adjust as needed)
    time.sleep(6)

    # Stop if we've collected 1000 articles
    if len(articles) >= 1000:
        articles = articles[:1000]
        break

# Create a DataFrame with the desired columns and shape (1000 rows, 3 columns)
df = pd.DataFrame(articles, columns=['title', 'link', 'description'])
df.to_csv('nyt_data.csv', index=False)
print("Data saved to nyt_data.csv")

# If you're running in Google Colab, this code will prompt a file download:
try:
    from google.colab import files
    files.download('nyt_data.csv')
except ImportError:
    print("Not running in Colab; please locate the 'nyt_data.csv' file in your working directory.")


Fetched page 1, total articles so far: 10
Fetched page 2, total articles so far: 20
Fetched page 3, total articles so far: 30
Fetched page 4, total articles so far: 40
Fetched page 5, total articles so far: 50
Error on page 5: 429
Data saved to nyt_data.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>